In [1]:
import pickle
import math
import numpy as np
import time
from collections import defaultdict

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
AutoML_path = ""
log_folder = "tpot_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022_tpot'

combination_name = 'DDN+BO'
today = '03102022'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_'+today


In [3]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))

pipeline_steps_class = [i for i in pipeline_steps_class if i[3] != "Timeout" and not np.isinf(i[3])]
print(len(pipeline_steps_class))

39
12863
11797


In [4]:
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor

In [5]:
target_test_pred_time = {}
data_names = list(DDN.keys())
data_embed = DDN

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len_task_train == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        X_train = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        y_train = np.array([i[3] for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        X_test = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        y_test = np.array([i[3] for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

        model = GaussianProcessRegressor(
            kernel=Matern(nu=2.5),alpha=1e-6,normalize_y=True,
            n_restarts_optimizer=0,random_state=1
        )
        
        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(X_train, y_train)
        end = time.time()
        duration = end-start

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print("Threshold", threshold, "MAE:", mae, "MSE:", mse)
        print("Time:", duration)
        target_test_pred_time[dict_key] = (y_test, y_pred, duration, mae, mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
X_train shape: (2129, 96) y_train shape: (2129,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.05 MAE: 0.04068036440968191 MSE: 0.0035708985979063957
Time: 1.64851713180542
X_train shape: (4352, 96) y_train shape: (4352,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.1 MAE: 0.04112425511040869 MSE: 0.0036557939409276982
Time: 9.697368860244751
X_train shape: (7632, 96) y_train shape: (7632,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.15000000000000002 MAE: 0.03969015779460519 MSE: 0.0034029156855455906
Time: 38.23489284515381
X_train shape: (8814, 96) y_train shape: (8814,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.2 MAE: 0.04410337226998233 MSE: 0.004320215955050672
Time: 53.70146608352661
X_train shape: (10228, 96) y_train shape: (10228,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.25 MAE: 0.03721130583978505 MSE: 0.003119571189121888
Time: 79.83213019371033
X_train shape: (10830, 96) y_train shape: (10830,)
X_te

Threshold 0.1 MAE: 0.03628731392569186 MSE: 0.003263096412001157
Time: 11.386152029037476
X_train shape: (8336, 96) y_train shape: (8336,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.15000000000000002 MAE: 0.029976077796605122 MSE: 0.002335889985077522
Time: 50.42848515510559
X_train shape: (8771, 96) y_train shape: (8771,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.2 MAE: 0.03303878929453345 MSE: 0.001998281891990248
Time: 57.418919801712036
X_train shape: (10185, 96) y_train shape: (10185,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.25 MAE: 0.04724586985874242 MSE: 0.0028503113969570596
Time: 77.568204164505
X_train shape: (10787, 96) y_train shape: (10787,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.3 MAE: 0.05632727097645922 MSE: 0.0037431830784916175
Time: 93.77089309692383
Australian_DDN+BO_0.3 Australian_DDN+BO_0.35
35 35
X_train shape: (11130, 96) y_train shape: (11130,)
X_test shape: (314, 96) y_test shape: (314,)
Threshol

Threshold 0.25 MAE: 0.08897474601304754 MSE: 0.008936540177512595
Time: 52.159059047698975
X_train shape: (9926, 96) y_train shape: (9926,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.3 MAE: 0.11747775314774568 MSE: 0.014828574051240159
Time: 75.7399160861969
X_train shape: (10492, 96) y_train shape: (10492,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.35000000000000003 MAE: 0.13022269294260091 MSE: 0.017985501323820416
Time: 86.18956708908081
X_train shape: (10813, 96) y_train shape: (10813,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.4 MAE: 0.1365097778688832 MSE: 0.01966247102040932
Time: 91.85489702224731
X_train shape: (11156, 96) y_train shape: (11156,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.45 MAE: 0.1450766204233651 MSE: 0.02207477736006265
Time: 96.3884060382843
11 connect-4
X_train shape: (2073, 96) y_train shape: (2073,)
X_test shape: (322, 96) y_test shape: (322,)
Threshold 0.05 MAE: 0.06807774350080047 MSE: 0.014625

X_train shape: (10920, 96) y_train shape: (10920,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.3 MAE: 0.3661254687912101 MSE: 0.17860970397540007
Time: 80.2453031539917
X_train shape: (11401, 96) y_train shape: (11401,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.35000000000000003 MAE: 0.3671170954911045 MSE: 0.17933680687964146
Time: 79.08916521072388
dionis_DDN+BO_0.35 dionis_DDN+BO_0.4
37 37
X_train shape: (11635, 96) y_train shape: (11635,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.45 MAE: 0.3557139033049712 MSE: 0.17109422608227512
Time: 87.85817217826843
16 fabert
X_train shape: (1175, 96) y_train shape: (1175,)
X_test shape: (239, 96) y_test shape: (239,)
Threshold 0.05 MAE: 0.08936861568592543 MSE: 0.008838162473888597
Time: 0.380918025970459
X_train shape: (3189, 96) y_train shape: (3189,)
X_test shape: (239, 96) y_test shape: (239,)
Threshold 0.1 MAE: 0.06552859343653782 MSE: 0.005954674480215443
Time: 3.3225491046905518
X_train shape:

X_train shape: (4452, 96) y_train shape: (4452,)
X_test shape: (302, 96) y_test shape: (302,)
Threshold 0.15000000000000002 MAE: 0.06106140226261077 MSE: 0.006209764872230024
Time: 8.968472957611084
jannis_DDN+BO_0.15 jannis_DDN+BO_0.2
15 15
X_train shape: (6971, 96) y_train shape: (6971,)
X_test shape: (302, 96) y_test shape: (302,)
Threshold 0.25 MAE: 0.08158438151493065 MSE: 0.010041527237751265
Time: 24.977782011032104
X_train shape: (7946, 96) y_train shape: (7946,)
X_test shape: (302, 96) y_test shape: (302,)
Threshold 0.3 MAE: 0.09589555801386218 MSE: 0.012794416643094388
Time: 31.613651037216187
X_train shape: (9715, 96) y_train shape: (9715,)
X_test shape: (302, 96) y_test shape: (302,)
Threshold 0.35000000000000003 MAE: 0.10789337412854902 MSE: 0.01563785164628305
Time: 55.494585037231445
X_train shape: (11495, 96) y_train shape: (11495,)
X_test shape: (302, 96) y_test shape: (302,)
Threshold 0.4 MAE: 0.1277646780125503 MSE: 0.021421716012358476
Time: 87.73452377319336
jannis

Threshold 0.3 MAE: 0.13538848987525468 MSE: 0.01911980901309133
Time: 53.10924816131592
X_train shape: (8723, 96) y_train shape: (8723,)
X_test shape: (362, 96) y_test shape: (362,)
Threshold 0.35000000000000003 MAE: 0.1473902093050554 MSE: 0.022381260843714897
Time: 49.44254207611084
X_train shape: (10137, 96) y_train shape: (10137,)
X_test shape: (362, 96) y_test shape: (362,)
Threshold 0.4 MAE: 0.1797387964445565 MSE: 0.03287760960098591
Time: 74.81512975692749
X_train shape: (10739, 96) y_train shape: (10739,)
X_test shape: (362, 96) y_test shape: (362,)
Threshold 0.45 MAE: 0.19091091047186612 MSE: 0.03708075450731722
Time: 93.83704495429993
27 mfeat-factors
X_train shape: (2771, 96) y_train shape: (2771,)
X_test shape: (279, 96) y_test shape: (279,)
Threshold 0.05 MAE: 0.0303365929077682 MSE: 0.0027186848600460854
Time: 3.0899157524108887
X_train shape: (4011, 96) y_train shape: (4011,)
X_test shape: (279, 96) y_test shape: (279,)
Threshold 0.1 MAE: 0.03713125993013302 MSE: 0.0028

Threshold 0.35000000000000003 MAE: 0.07609304589913116 MSE: 0.0068655955534286574
Time: 106.7916169166565
phoneme_DDN+BO_0.35 phoneme_DDN+BO_0.4
36 36
X_train shape: (11237, 96) y_train shape: (11237,)
X_test shape: (326, 96) y_test shape: (326,)
Threshold 0.45 MAE: 0.07861797353389656 MSE: 0.007473524411324029
Time: 107.72940492630005
32 riccardo
X_train shape: (3386, 96) y_train shape: (3386,)
X_test shape: (217, 96) y_test shape: (217,)
Threshold 0.05 MAE: 0.053597327922874013 MSE: 0.006098040361142329
Time: 5.027281045913696
X_train shape: (4073, 96) y_train shape: (4073,)
X_test shape: (217, 96) y_test shape: (217,)
Threshold 0.1 MAE: 0.05352478880141108 MSE: 0.00551968332616436
Time: 7.579390048980713
X_train shape: (6473, 96) y_train shape: (6473,)
X_test shape: (217, 96) y_test shape: (217,)
Threshold 0.15000000000000002 MAE: 0.06166869095477413 MSE: 0.0053879180912585626
Time: 25.82051706314087
riccardo_DDN+BO_0.15 riccardo_DDN+BO_0.2
20 20
X_train shape: (7732, 96) y_train sh

X_train shape: (2016, 96) y_train shape: (2016,)
X_test shape: (379, 96) y_test shape: (379,)
Threshold 0.05 MAE: 0.0639830690855572 MSE: 0.007362524846510945
Time: 2.0596730709075928
X_train shape: (3430, 96) y_train shape: (3430,)
X_test shape: (379, 96) y_test shape: (379,)
Threshold 0.1 MAE: 0.08342684970060547 MSE: 0.00989833788807846
Time: 5.556883811950684
X_train shape: (6111, 96) y_train shape: (6111,)
X_test shape: (379, 96) y_test shape: (379,)
Threshold 0.15000000000000002 MAE: 0.05825585115996114 MSE: 0.007607797081171186
Time: 21.233567237854004
X_train shape: (6751, 96) y_train shape: (6751,)
X_test shape: (379, 96) y_test shape: (379,)
Threshold 0.2 MAE: 0.05795676034646445 MSE: 0.0076617889813374454
Time: 26.402743101119995
X_train shape: (9068, 96) y_train shape: (9068,)
X_test shape: (379, 96) y_test shape: (379,)
Threshold 0.25 MAE: 0.062187189005780935 MSE: 0.010329812328710733
Time: 72.52208518981934
X_train shape: (11065, 96) y_train shape: (11065,)
X_test shape: